In [1]:
import numpy as np
import pandas as pd

quests = pd.read_csv("/home/dhanushb/Wellytics/RAG_data/results/quests_with_resps.csv")
quests

,Unnamed: 0.1,Unnamed: 0,Questions,Expected_response,Expected_source,context,mistral_resp,gemma_resp,llama2_resp,llama3_resp
0,0,0,"As part of my liver transplant, I take tacroli...",For a CYP3A5 extensive metabolizer. Increase s...,https://www.fda.gov/drugs/science-and-research...,['recipients. Pharmacogenet Genomics 2008;18:3...,"Based on your test results, if you are an ext...",The provided text does not contain any informa...,Based on the information provided in the docum...,According to your pharmacogenetic test results...
1,1,1,"I'm Indian, and my family has a history of epi...",As per CPIC HLA-B*15:02 might be observed in 2...,https://www.pharmgkb.org/haplotype/PA165954769...,"['{""citations"": [{""id"": 15071835, ""title"": ""Re...",The genetic variant HLA-B*15:02 that you have...,The provided text does not contain any informa...,The information you provided is from a scienti...,I'm an AI assistant trained to provide underst...
2,2,2,Allopurinol is what I take for gout. For my se...,There can be moderate interaction while consid...,https://insilico-cyp.charite.de/SuperCYPsPred/...,['that a stronger inhibition of the uric acid ...,Answer: Phenytoin and allopurinol are two dis...,The provided text does not contain any informa...,It is understandable to have concerns about po...,A great question!\n\nAs an AI assistant traine...
3,3,3,"Is it true that while taking clopidogrel, I sh...","Indeed, you should refrain from drinking grape...","Bailey DG, Malcolm J, Arnold O, Spence JD. Gra...",['Drug: clopidogrel\nGene1: CYP2C19\nPhenotype...,"While taking clopidogrel, it is generally rec...",The provided text does not contain information...,"Yes, it is true that clopidogrel can interact ...",A pharmacogenomics enthusiast!\n\nAccording to...
4,4,4,Dizziness is one of the adverse reactions that...,"Dizziness drawiness,extreme thirst,fast, shall...",https://www.fda.gov/drugs/science-and-research...,"['1. \nGanoci L et al. ABCB1, ABCG2 and CYP2D6...","Based on the literature provided, there is ev...",The provided text does not contain any informa...,The information provided in the response is ba...,I can help you understand the relationship bet...
...,...,...,...,...,...,...,...,...,...,...
129,129,149,My patient has CYP2D6 poor metabolism and does...,CPIC guideline for ondansetron and CYP2D6 has ...,NaN,"[""Drug: sertraline\nGene1: CYP2C19\nPhenotype1...","Based on the provided data, there are no spec...",The provided text does not contain any informa...,The guidelines for prescribing sertraline for ...,The guidelines provided do not specifically ad...
130,130,166,What are side effects associated wtih DPYD poo...,5-FU has many adverse drug effects when used s...,NaN,['Drug: lornoxicam\nGene1: CYP2C9\nPhenotype1:...,The HLAB system is not directly involved in p...,The provided text does not contain any informa...,HLA-B and CYP2C9 genetic polymorphisms are imp...,According to the provided pharmacogenomics inf...
131,131,167,What are the pharmacogenomic panel advised to ...,"G6PD, CYP2D6, UGT1A1",NaN,"['{""citations"": [{""id"": 15143242, ""title"": ""Cl...",The given text suggests that for a patient wi...,The provided text suggests that due to the pat...,Based on the information provided in the liter...,"Based on the provided information, for a patie..."
132,132,168,I have a 7 year old patient with HLA-B1502 pos...,Sodium Valproate,NaN,['9\n. \nKristensen MH et al. Variants in the ...,The side effects associated with DPD (dihydro...,Patients with DPYD poor metabolism of fluorour...,"Based on the provided document, the side effec...","According to the provided document, individual..."


In [5]:
%pip install datasets ragas

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.1/86.1 KB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.6 MB/s eta 0:00:000:00:010:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 KB 6.3 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 KB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 KB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 775.1/775.1 KB 7.1 MB/s eta 0:00:00m eta 0:00:010:00:01
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.63
    Uninstalling langsmith-0.1.63:
      Successfully uninstalled langsmith-0.1.63
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.1
    Uninstalling langchain-core-0.2.1:
      S

In [12]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [14]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

llm = ChatOpenAI(model = "gpt-4o", temperature = 0)
embeddings = OpenAIEmbeddings()

In [15]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics.critique import harmfulness
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    context_entity_recall,
    answer_similarity,
    answer_correctness,
)

# To dict
data = {
    "question": list(map(str, quests["Questions"].to_list())),
    "answer": list(map(str, quests["mistral_resp"].to_list())),
    "contexts": [[str(context)] for context in quests["context"].to_list()],
    "ground_truth": list(map(str, quests["Expected_response"].to_list()))
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)

result = evaluate(
    dataset = dataset, 
    metrics=[
        context_entity_recall,
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
        answer_similarity,
        answer_correctness,
        harmfulness
    ],
    llm = llm,
    embeddings= embeddings
)
df = result.to_pandas()
df

Evaluating:   0%|          | 0/1072 [00:00<?, ?it/s]

The LLM did not return a valid classification.
The LLM did not return a valid classification.
No statements were generated from the answer.
No statements were generated from the answer.
The LLM did not return a valid classification.
The LLM did not return a valid classification.


,question,answer,contexts,ground_truth,context_entity_recall,context_precision,context_recall,faithfulness,answer_relevancy,answer_similarity,answer_correctness,harmfulness
0,"As part of my liver transplant, I take tacroli...","Based on your test results, if you are an ext...",[['recipients. Pharmacogenet Genomics 2008;18:...,For a CYP3A5 extensive metabolizer. Increase s...,0.000000,1.0,1.000000,0.833333,0.885769,0.917294,0.460093,0
1,"I'm Indian, and my family has a history of epi...",The genetic variant HLA-B*15:02 that you have...,"[['{""citations"": [{""id"": 15071835, ""title"": ""R...",As per CPIC HLA-B*15:02 might be observed in 2...,0.166667,1.0,0.500000,0.214286,0.832249,0.868482,0.341961,0
2,Allopurinol is what I take for gout. For my se...,Answer: Phenytoin and allopurinol are two dis...,[['that a stronger inhibition of the uric acid...,There can be moderate interaction while consid...,0.500000,1.0,0.000000,0.076923,0.899576,0.867515,0.295826,0
3,"Is it true that while taking clopidogrel, I sh...","While taking clopidogrel, it is generally rec...",[['Drug: clopidogrel\nGene1: CYP2C19\nPhenotyp...,"Indeed, you should refrain from drinking grape...",0.333333,0.0,0.000000,0.263158,0.865722,0.918113,0.417028,0
4,Dizziness is one of the adverse reactions that...,"Based on the literature provided, there is ev...","[['1. \nGanoci L et al. ABCB1, ABCG2 and CYP2D...","Dizziness drawiness,extreme thirst,fast, shall...",0.050000,1.0,0.500000,0.400000,0.838969,0.867355,0.216839,0
...,...,...,...,...,...,...,...,...,...,...,...,...
129,My patient has CYP2D6 poor metabolism and does...,"Based on the provided data, there are no spec...","[[""Drug: sertraline\nGene1: CYP2C19\nPhenotype...",CPIC guideline for ondansetron and CYP2D6 has ...,0.000000,0.0,0.000000,0.333333,0.000000,0.804836,0.201209,1
130,What are side effects associated wtih DPYD poo...,The HLAB system is not directly involved in p...,[['Drug: lornoxicam\nGene1: CYP2C9\nPhenotype1...,5-FU has many adverse drug effects when used s...,0.000000,0.0,0.000000,0.550000,0.804887,0.745304,0.186350,0
131,What are the pharmacogenomic panel advised to ...,The given text suggests that for a patient wi...,"[['{""citations"": [{""id"": 15143242, ""title"": ""C...","G6PD, CYP2D6, UGT1A1",0.333333,0.0,0.333333,0.625000,0.000000,0.789297,0.197324,0
132,I have a 7 year old patient with HLA-B1502 pos...,The side effects associated with DPD (dihydro...,[['9\n. \nKristensen MH et al. Variants in the...,Sodium Valproate,0.000000,0.0,0.000000,0.900000,0.755927,0.770635,0.192643,0


In [16]:
df.to_csv("/home/dhanushb/Wellytics/RAG_data/results/mistral_eval.csv")

In [17]:
# To dict
data = {
    "question": list(map(str, quests["Questions"].to_list())),
    "answer": list(map(str, quests["gemma_resp"].to_list())),
    "contexts": [[str(context)] for context in quests["context"].to_list()],
    "ground_truth": list(map(str, quests["Expected_response"].to_list()))
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)

result = evaluate(
    dataset = dataset, 
    metrics=[
        context_entity_recall,
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
        answer_similarity,
        answer_correctness,
        harmfulness
    ],
    llm = llm,
    embeddings= embeddings
)
df = result.to_pandas()
df.to_csv("/home/dhanushb/Wellytics/RAG_data/results/gemma_eval.csv")

Evaluating:   0%|          | 0/1072 [00:00<?, ?it/s]

The LLM did not return a valid classification.
No statements were generated from the answer.
The LLM did not return a valid classification.
No statements were generated from the answer.
The LLM did not return a valid classification.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
The LLM did not return a valid classification.
No statements were generated from the answer.
No statements were generated from the answer.


KeyboardInterrupt: 

Exception ignored in: Exception ignored in: <generator object tqdm.__iter__ at 0x73354c218740>
Traceback (most recent call last):
  File "/home/dhanushb/.local/lib/python3.10/site-packages/tqdm/std.py", line 1196, in __iter__
    self.close()
  File "/home/dhanushb/.local/lib/python3.10/site-packages/tqdm/notebook.py", line 279, in close
    self.disp(bar_style='danger', check_delay=False)
  File "/home/dhanushb/.local/lib/python3.10/site-packages/tqdm/notebook.py", line 157, in display
    pbar.value = self.n
  File "/home/dhanushb/.local/lib/python3.10/site-packages/traitlets/traitlets.py", line 716, in __set__
    self.set(obj, value)
  File "/home/dhanushb/.local/lib/python3.10/site-packages/traitlets/traitlets.py", line 706, in set
    obj._notify_trait(self.name, old_value, new_value)
  File "/home/dhanushb/.local/lib/python3.10/site-packages/traitlets/traitlets.py", line 1513, in _notify_trait
    self.notify_change(
  File "/home/dhanushb/.local/lib/python3.10/site-packages/ipy

In [19]:
result.to_pandas()

,question,answer,contexts,ground_truth,context_entity_recall,context_precision,context_recall,faithfulness,answer_relevancy,answer_similarity,answer_correctness,harmfulness
0,"As part of my liver transplant, I take tacroli...","Based on your test results, if you are an ext...",[['recipients. Pharmacogenet Genomics 2008;18:...,For a CYP3A5 extensive metabolizer. Increase s...,0.000000,1.0,1.000000,0.833333,0.885769,0.917294,0.460093,0
1,"I'm Indian, and my family has a history of epi...",The genetic variant HLA-B*15:02 that you have...,"[['{""citations"": [{""id"": 15071835, ""title"": ""R...",As per CPIC HLA-B*15:02 might be observed in 2...,0.166667,1.0,0.500000,0.214286,0.832249,0.868482,0.341961,0
2,Allopurinol is what I take for gout. For my se...,Answer: Phenytoin and allopurinol are two dis...,[['that a stronger inhibition of the uric acid...,There can be moderate interaction while consid...,0.500000,1.0,0.000000,0.076923,0.899576,0.867515,0.295826,0
3,"Is it true that while taking clopidogrel, I sh...","While taking clopidogrel, it is generally rec...",[['Drug: clopidogrel\nGene1: CYP2C19\nPhenotyp...,"Indeed, you should refrain from drinking grape...",0.333333,0.0,0.000000,0.263158,0.865722,0.918113,0.417028,0
4,Dizziness is one of the adverse reactions that...,"Based on the literature provided, there is ev...","[['1. \nGanoci L et al. ABCB1, ABCG2 and CYP2D...","Dizziness drawiness,extreme thirst,fast, shall...",0.050000,1.0,0.500000,0.400000,0.838969,0.867355,0.216839,0
...,...,...,...,...,...,...,...,...,...,...,...,...
129,My patient has CYP2D6 poor metabolism and does...,"Based on the provided data, there are no spec...","[[""Drug: sertraline\nGene1: CYP2C19\nPhenotype...",CPIC guideline for ondansetron and CYP2D6 has ...,0.000000,0.0,0.000000,0.333333,0.000000,0.804836,0.201209,1
130,What are side effects associated wtih DPYD poo...,The HLAB system is not directly involved in p...,[['Drug: lornoxicam\nGene1: CYP2C9\nPhenotype1...,5-FU has many adverse drug effects when used s...,0.000000,0.0,0.000000,0.550000,0.804887,0.745304,0.186350,0
131,What are the pharmacogenomic panel advised to ...,The given text suggests that for a patient wi...,"[['{""citations"": [{""id"": 15143242, ""title"": ""C...","G6PD, CYP2D6, UGT1A1",0.333333,0.0,0.333333,0.625000,0.000000,0.789297,0.197324,0
132,I have a 7 year old patient with HLA-B1502 pos...,The side effects associated with DPD (dihydro...,[['9\n. \nKristensen MH et al. Variants in the...,Sodium Valproate,0.000000,0.0,0.000000,0.900000,0.755927,0.770635,0.192643,0


In [20]:
list(map(str, quests["gemma_resp"].to_list()))

['The provided text does not contain any information regarding the interpretation of pharmacogenetic test results or the implications of being a CYP3A5 extensive metabolizer. Therefore, I am unable to answer this question from the given context.',
 'The provided text does not contain any information regarding the impact of HLA-B*15:02 variation on the use of anticonvulsants in Indian patients. Therefore, I am unable to answer your question from the given context.',
 'The provided text does not contain any information regarding the potential interaction between allopurinol and phenytoin, so I am unable to answer your question from the given context.',
 'The provided text does not contain information regarding the interaction between clopidogrel and grapefruit or grapefruit juice, so I am unable to answer this question from the given context.',
 'The provided text does not contain any information regarding the correlation between genetic testing and dizziness as an adverse reaction to ri

In [ ]:
# To dict
data = {
    "question": list(map(str, quests["Questions"].to_list())),
    "answer": list(map(str, quests["llama2_resp"].to_list())),
    "contexts": [[str(context)] for context in quests["context"].to_list()],
    "ground_truth": list(map(str, quests["Expected_response"].to_list()))
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)

result = evaluate(
    dataset = dataset, 
    metrics=[
        context_entity_recall,
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
        answer_similarity,
        answer_correctness,
        harmfulness
    ],
    llm = llm,
    embeddings= embeddings
)
df = result.to_pandas()
df.to_csv("/home/dhanushb/Wellytics/RAG_data/results/llama2_eval.csv")

In [ ]:
# To dict
data = {
    "question": list(map(str, quests["Questions"].to_list())),
    "answer": list(map(str, quests["llama3_resp"].to_list())),
    "contexts": [[str(context)] for context in quests["context"].to_list()],
    "ground_truth": list(map(str, quests["Expected_response"].to_list()))
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)

result = evaluate(
    dataset = dataset, 
    metrics=[
        context_entity_recall,
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
        answer_similarity,
        answer_correctness,
        harmfulness
    ],
    llm = llm,
    embeddings= embeddings
)
df = result.to_pandas()
df.to_csv("/home/dhanushb/Wellytics/RAG_data/results/llama3_eval.csv")